## Reading the file(filtered from sql)


In [ ]:
import pandas as pd

df = pd.read_csv('ls_new.csv')

df

In [ ]:
items_to_remove = ['Paper Checkout Bags','Discount', 'Clearance']

filtered_df = df[~df['Details'].isin(items_to_remove)]

In [ ]:
filtered_df.info()

In [ ]:
filtered_df['Quantity'].sum

In [ ]:
# Ensure 'Quantity' is integer and handle NaNs
filtered_df['Quantity'] = filtered_df['Quantity'].fillna(0).astype(int)

# Repeat 'Details' by 'Quantity' to calculate the real item quantity sold in each row
repeated_details = filtered_df.apply(lambda row: [row['Details']] * row['Quantity'], axis=1)

# Flatten the list of lists
flattened_details = [item for sublist in repeated_details for item in sublist]

# Calculate value counts
details_counts = pd.Series(flattened_details).value_counts().reset_index()

details_counts.columns = ['NameofProduct', 'PurchaseFrequencyDuringHolidays']

In [ ]:
details_counts.columns = ['NameofProduct', 'PurchaseFrequencyDuringHolidays']

In [ ]:
details_counts.info()

In [ ]:
details_counts.to_clipboard(index=False)

In [ ]:
categories = pd.read_csv('product-export.csv')
categories

In [ ]:
categories_small = categories[['name','product_category']]

In [ ]:
details_counts['PurchaseFrequencyDuringHolidays'].sum()

In [ ]:
total_units_sold = details_counts['PurchaseFrequencyDuringHolidays'].sum()

print(f"Total units sold is: {total_units_sold}")

In [ ]:
details_counts.to_csv('NumberofUnitsSoldJantoMay.csv')

# Apriori Process

In [ ]:
sales_line_df = filtered_df[['ReceiptNumber', 'Details']]

In [ ]:
# Convert the transaction data into a one-hot encoded format
# Get unique items
unique_items = sales_line_df['Details'].unique()
# Transform to Table Format (One-Hot Encoding)
df_retail_txn_table = sales_line_df.pivot_table(index='ReceiptNumber', columns='Details', aggfunc='size', fill_value=0)
df_retail_txn_table

In [ ]:
# You can see we have the count, which may be >1 for some cases. We'll change this to binary to align with the format.
df_retail_txn_table = (df_retail_txn_table > 0).astype(int) # This one sets all 1+ values to True & convert it to 1
df_retail_txn_table

In [ ]:
# We will first convert the DataFrame to have Boolean (True/False) instead of 1/0
df_retail_final=(df_retail_txn_table > 0)
df_retail_final

In [ ]:
! pip install mlxtend

In [ ]:
# Import necessary libraries
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import seaborn as sns

In [ ]:
# Apply the Apriori algorithm to find frequent itemsets
min_support = 0.001  # Minimum support threshold - change this number and run again
frequent_itemsets = apriori(df_retail_final, min_support=min_support, use_colnames=True)
frequent_itemsets

In [ ]:
# Discover association rules
min_confidence = 0.01  # Minimum confidence threshold
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=min_confidence)
rules

In [ ]:
rules.head(50)

In [ ]:
# Let us compile a final list with some filter, calculation & sorting
final_rules=rules[(rules['lift']>1)&(rules['support']>=0.001)&(rules['confidence']>=0.01)]
# Determine number of items in X => predicting number of items in Y
final_rules=final_rules.copy() # this step creates an independent list instead of a view on rules from above
# We capture the number of items in the list of each antecedent/consequent set using 'len' function for each row
final_rules['antecedent_count']=final_rules['antecedents'].apply(len)
final_rules['consequent_count']=final_rules['consequents'].apply(len) #X

# Apply some rounding and sorting on Lift
final_rules=round(final_rules,2).sort_values(by=['lift', 'confidence'], ascending=[False, False])

# Preserve relevant columns only
final_rules=final_rules[['antecedents', 'antecedent_count', 'consequents', 'consequent_count', 'support', 'confidence', 'lift']]
final_rules

In [ ]:
final_rules.to_csv('final_rules_unfiltered.csv')

In [ ]:
for index, row in final_rules.head(100).iterrows():
  antecedents=' =+= '.join(list(row['antecedents']))
  consequents=' =+= '.join(list(row['consequents']))
  rule_statement='If customer buys '+antecedents+', they are '+str(row['confidence'] * 100)+ \
  '% likely to buy '+consequents+' - this is '+str(row['lift'])+' times more likely than random chance.'
  print(rule_statement)
